In [1]:
import itertools
import operator
import pandas as pd
import pickle
import numpy as np
import json
from tqdm import tqdm, trange
from time import sleep
import gc
import os
import morfeusz2 
import nltk
from dask import array as da
import re
import time
import itertools
from twitter_tools import tweet_stats, preprocess_tweets
from collections import OrderedDict
from sklearn.feature_extraction.text import TfidfVectorizer
from upsetplot import plot as setplot
from tweepy import API, AppAuthHandler, Cursor
from config import consumer_key, consumer_secret, access_token, access_secret
from twitter_scrapers import FollowerScraper, KeywordsScraper
from matplotlib import pyplot as plt
import datetime

In [4]:
tmp = pd.read_csv("data/clean/gov_tweets_clean_2020_02_22.csv")

In [7]:
tmp = tmp["created_at"].tolist()

In [9]:
import re

In [20]:
tmp[0] = ""

In [21]:
len(tmp)

24373

In [22]:
tmp2 = [i for i in tmp if re.match("[A-Z][a-z]{2} [A-Z][a-z]{2} \d{2} \d{2}:\d{2}:\d{2} \+\d{4} \d{4}", str(i))]

In [23]:
len(tmp)

24373

In [24]:
len(tmp2)

24372

In [4]:
auth = AppAuthHandler(consumer_key, consumer_secret)
api = API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [1]:
government_followers = json.load(open("data/government_followers.json","r"))
opposition_followers = json.load(open("data/opposition_followers.json","r"))

NameError: name 'json' is not defined

In [ ]:
def intersect_followers(followers_dict):
    profile_names = list(followers_dict.keys())
    followers_intersections = [pd.Series(True, index=list(elements), name=name)
                      for name, elements in followers_dict.items()] #set all values to true where id exists
    followers_intersections = pd.concat(followers_intersections,sort = False,axis = 1) #concatanate
    followers_intersections = followers_intersections.fillna(False).reset_index() #fill nas and put id to columns
    followers_intersections = followers_intersections.groupby(profile_names).count() #get counts
    return followers_intersections["index"]

In [ ]:
opposition_followers = {k:set(v) for k,v in opposition_followers.items()}
setplot(intersect_followers(opposition_followers),show_percentages = True)
plt.show()

In [ ]:
government_followers = {k:set(v) for k,v in government_followers.items()}
government_intersections = intersect_followers(government_followers)
setplot(government_intersections,show_percentages = True)
plt.show()
print(pd.DataFrame(government_intersections).sort_values(by = "index"))

### Get followers of politician profiles
The method `getFollowers` from `FollowerScrapers` given a list of Twitter account names returns dictionary of the account names from the list associatied with list of IDs of their followers. 

**Get followers of government politicians profiles**

In [24]:
if os.path.isfile("data/government_followers.json"):
    government_followers = json.load(open("data/government_followers.json","r"))
else:
    government = ['@D_Tarczynski', '@BeataSzydlo', '@Macierewicz_A', 
                  '@KrystPawlowicz', '@StKarczewski', '@MorawieckiM', 
                  '@ZiobroPL', '@jbrudzinski', '@PatrykJaki', '@mblaszczak']
    government_followers = scraper.getFollowers(government, "data/government_followers.json")

**Get followers of opposition politicians profiles**

In [25]:
if os.path.isfile("data/opposition_followers.json"):
    opposition_followers = json.load(open("data/opposition_followers.json","r"))
else:
    opposition = ['@SchetynadlaPO', '@bbudka', '@KLubnauer', '@Arlukowicz', 
                  '@profGrodzki', '@RyszardPetru', '@trzaskowski_', 
                  '@TomaszSiemoniak', '@Gasiuk_Pihowicz']
    opposition_followers = scraper.getFollowers(opposition, "data/opposition_followers.json")

### Narrow down the population to political partisans
The method `subsetFollowers` given two followers dictionaries such as the ones created by the `getFollowers` method returns two lists of followers, each that had at least $n$ overlap within one dictionary and at most $m$ overlap with the other. Using this method I filtered out two distinct populations of users - *opposition partisans* following at least 6 opposition politicians and at most 4 government politicians and *government partisans* following at least 6 government politicians and and most.

In [13]:
if os.path.isfile("data/gov_partisans.pickle") and os.path.isfile("data/opp_partisans.pickle"):
    gov_partisans = pickle.load(open("data/gov_partisans.pickle","rb"))
    opp_partisans = pickle.load(open("data/opp_partisans.pickle","rb"))
else:
    gov_partisans, opp_partisans = scraper.subsetFollowers(government_followers, opposition_followers, 6, 4)
    pickle.dump(gov_partisans,open("data/gov_partisans.pickle","wb"))
    pickle.dump(opp_partisans,open("data/opp_partisans.pickle","wb"))

The size of these groups:

In [14]:
print("{} opposition partisans and {} government partisans were filtered".format(len(gov_partisans),len(opp_partisans)))

35422 opposition partisans and 26465 government partisans were filtered


As expected, there's no overlap between the two groups:

In [ ]:
set(gov_partisans).intersection(set(opp_partisans))

### Get population info

In [21]:
if os.path.isfile("data/gov_partisans_info.csv"):
    gov_followers_info = pd.read_csv("data/gov_partisans_info.csv", index_col = 0)
else:
    scraper.getFollowersData(gov_partisans, "data/gov_partisans_info.csv")

In [22]:
if os.path.isfile("data/opp_partisans_info.csv"):
    opp_followers_info = pd.read_csv("data/opp_partisans_info.csv", index_col = 0)
else:
    scraper.getFollowersData(opp_partisans, "data/opp_partisans_info.csv")

100%|██████████| 26465/26465 [7:10:03<00:00,  1.03it/s]    


### Get samples:
Given the above partisans, I sampled 5000 from each group that met the following constraints:
- every one had at least 1000 tweets in life time
- they were active within last 7 days (01.04.2020) 
- the Twitter language was "pl" or "None"

In [ ]:
if os.path.isfile("data/gov_sample.pickle"):
    gov_sample = pickle.load(open("data/gov_sample.pickle","rb"))
else:
    gov_sample = scraper.sampleFollowers(gov_partisans, num_users = 5000, min_usr_tweets = 1000, 
                            last_date_active = datetime.datetime.now() - datetime.timedelta(days = 7),
                           lang = [None,"pl"])
    pickle.dump(gov_sample,open("data/gov_sample.pickle","wb"))

In [ ]:
if os.path.isfile("data/opp_sample.pickle"):
    opp_sample = pickle.load(open("data/opp_sample.pickle","rb"))
else:
    opp_sample = scraper.sampleFollowers(opp_partisans, num_users = 5000, min_usr_tweets = 1000, 
                            last_date_active = datetime.datetime.now() - datetime.timedelta(days = 7),
                           lang = [None,"pl"])
    pickle.dump(opp_sample,open("data/opp_sample.pickle","wb"))

In [5]:
scraper = FollowerScraper(api = api, path_log = "test/scraper_test.log")
twitter_columns = pickle.load(open("data/twitter_columns.pickle","rb"))

### Scrape
Scraping ids selected based on the above conditions that were active in March .

In [6]:
gov_ids = pickle.load(open("data/sample/gov_ids.pickle","rb"))

**Load full (approximately) list of possible columns that can be scraped:**

In [3]:
twitter_columns = pickle.load(open("twitter_columns.pickle","rb"))

**Initialize scraper:**

In [4]:
scraper = FollowerScraper(api = api, path_log = "scraper.log")

#### Government

In [4]:
gov_ids = pickle.load(open("data/sample/gov_ids.pickle","rb"))

In [22]:
scraper.scrape(gov_ids[0:5000],path = "data/gov_tweets",
               min_date = datetime.datetime.strptime("22/02/2020","%d/%m/%Y"),
               limit = 50, cols = twitter_columns)

100%|██████████| 5000/5000 [21:33:22<00:00, 15.52s/it]   


Total tweets scraped 1692598


In [8]:
scrape_from = datetime.datetime.now() - datetime.timedelta(days = 5)
scraper.scrape(gov_ids[0:5000],path = "data/scraped/gov_tweets",
               min_date = scrape_from,
               limit = 50, cols = twitter_columns)

100%|██████████| 5000/5000 [5:59:28<00:00,  4.31s/it]   


Total tweets scraped 63269


#### Opposition

In [5]:
opp_ids = pickle.load(open("data/sample/opp_ids.pickle","rb"))

In [ ]:
scraper.scrape(opp_ids[0:5000], path = "data/opp_tweets",
               min_date = datetime.datetime.strptime("22/02/2020","%d/%m/%Y"), 
               limit = 50, cols = twitter_columns)

In [9]:
scrape_from = datetime.datetime.now() - datetime.timedelta(days = 5)
scraper.scrape(opp_ids[:5000], path = "data/scraped/opp_tweets",
               min_date = scrape_from, 
               limit = 50, cols = twitter_columns)

100%|██████████| 5000/5000 [5:40:38<00:00,  4.09s/it]   


Total tweets scraped 52434


Fala sprzeciwu:

In [7]:
fala_sprzeciwu = "#falasprzeciwu OR #FalaSprzeciwu OR #FalaSprzeciwuLive OR #CzarnyProtest OR #czarnyprotest"
scraper = KeywordsScraper(api = api, path_log = "scraper_fala.log")
scraper.scrape(fala_sprzeciwu, min_date = datetime.datetime.now() - datetime.timedelta(days = 14), 
               path = "data/fala", limit = 50, cols = twitter_columns)

Total tweets scraped 115


### Trash

In [26]:
#function to filter out elements of superset that occur in at most/least n subsets
def how_many(superset,subsets):
    superset = np.array(superset)[:,np.newaxis] #convert to Nx1 nparray
    superset = da.from_array(superset, chunks = (2000,1)) #convert to dask
    count = np.zeros(superset.shape[0]).astype("int8")
    for subset in tqdm(subsets):
        tmp = da.from_array(subset)
        count += (superset == tmp).sum(axis = 1).compute().astype("int8")
        gc.collect()
    return count

government_superset = list(set(itertools.chain.from_iterable(government_followers.values())))
government_subsets = list(government_followers.values())
opposition_superset = list(set(itertools.chain.from_iterable(opposition_followers.values())))
opposition_subsets = list(opposition_followers.values())


gov_in_gov = how_many(government_superset, government_subsets) #how many gov profiles followed by each gov follower
gov_in_opp = how_many(government_superset, opposition_subsets) #how many opp profiles followed by each gov follower
opp_in_opp = how_many(opposition_superset, opposition_subsets) #how many opp profiles followed by each opp follower
opp_in_gov = how_many(opposition_superset, government_subsets) #how many gov profiles followed by each opp follower
pickle.dump([gov_in_gov,gov_in_opp,opp_in_opp,opp_in_gov], open("overlaps.pickle","wb"))

    
"""    
gov_partisans = np.array(government_superset)[(gov_in_gov >= 6) & (gov_in_opp <= 4)].tolist()
opp_partisans = np.array(opposition_superset)[(opp_in_opp >= 6) & (opp_in_gov <= 4)].tolist()


pickle.dump(gov_partisans, open("data/gov_partisans.pickle","wb"))
pickle.dump(opp_partisans, open("data/opp_partisans.pickle","wb"))


#no overlap
set(gov_partisans).intersection(set(opp_partisans))
"""

100%|██████████| 10/10 [12:11<00:00, 73.17s/it]


'    \ngov_partisans = np.array(government_superset)[(gov_in_gov >= 6) & (gov_in_opp <= 4)].tolist()\nopp_partisans = np.array(opposition_superset)[(opp_in_opp >= 6) & (opp_in_gov <= 4)].tolist()\n\n\npickle.dump(gov_partisans, open("data/gov_partisans.pickle","wb"))\npickle.dump(opp_partisans, open("data/opp_partisans.pickle","wb"))\n\n\n#no overlap\nset(gov_partisans).intersection(set(opp_partisans))\n'

In [27]:
gov_partisans = np.array(government_superset)[(gov_in_gov > gov_in_opp)].tolist()
opp_partisans = np.array(opposition_superset)[(opp_in_opp > opp_in_gov)].tolist()

In [31]:
len(gov_partisans)

452429

In [32]:
len(opp_partisans)

440215

In [ ]:
!rm test/gov_tweets.json
!rm test/gov_tweets.csv
del(scraper)
auth = AppAuthHandler(consumer_key, consumer_secret)
api = API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)
scraper = FollowerScraper(api = api, path_log = "test/scraper_test.log")
scraper.scrape(gov_ids[0:10],path = "test/gov_tweets",
               min_date = datetime.datetime.strptime("28/04/2020","%d/%m/%Y"),
               limit = 50, cols = twitter_columns)
temp = json.load(open("test/gov_tweets.json","r"))
print(len(temp['3257053341']))
temp['3257053341'] = test
json.dump(temp,open("test/gov_tweets.json","w"))
print(len(temp['3257053341']))